# AOC 2021

Welcome to the Advent of Code 2021 !

## Basic configuration



In [1]:
!pip install aocd

     |████████████████████████████████| 97 kB 3.2 MB/s eta 0:00:011
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
     |████████████████████████████████| 74 kB 1.8 MB/s  eta 0:00:01
  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
     |████████████████████████████████| 61 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 339 kB 20.3 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=25031920a8d9522dbf31f0146a2207f6853793c9052016e69f277be49f49dce7
  Stored in directory: /home/naomi/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor


In [ ]:
import os

os.environ['AOC_SESSION'] = #your login session cookie

In [ ]:
from aocd import submit
from aocd.models import Puzzle

In [2]:
import os 
import numpy as np
import tqdm
import json

## Day 24
https://adventofcode.com/2021/day/24
### Part 1

### Part 2

## Day 23
https://adventofcode.com/2021/day/23
### Part 1

### Part 2

## Day 22
https://adventofcode.com/2021/day/22
### Part 1

### Part 2

## Day 21
https://adventofcode.com/2021/day/21
### Part 1

### Part 2

## Day 20
https://adventofcode.com/2021/day/20
### Part 1

### Part 2

## Day 19
https://adventofcode.com/2021/day/19
### Part 1

### Part 2

## Day 18
https://adventofcode.com/2021/day/18
### Part 1

### Part 2

## Day 17
https://adventofcode.com/2021/day/17
### Part 1

### Part 2

## Day 16
https://adventofcode.com/2021/day/16
### Part 1

### Part 2

## Day 15
https://adventofcode.com/2021/day/15

### Part 1

### Part 2

## Day 14
https://adventofcode.com/2021/day/14
### Part 1

### Part 2

## Day 13
https://adventofcode.com/2021/day/13
### Part 1

### Part 2

## Day 12
https://adventofcode.com/2021/day/12
### Part 1

### Part2

## Day 11
https://adventofcode.com/2021/day/11
### Part 1

### Part 2

## Day 10
https://adventofcode.com/2021/day/10
### Part 1

### Part 2

## Day 9
https://adventofcode.com/2021/day/9
### Part 1

### Part 2

## Day 8
https://adventofcode.com/2021/day/8
### Part 1

### Part2

## Day 7
https://adventofcode.com/2021/day/7
### Part 1

### Part 2

## Day 6
https://adventofcode.com/2021/day/6
### Part 1

### Part 2

## Day 5
https://adventofcode.com/2021/day/5
### Part 1

### Part 2

## Day 4
https://adventofcode.com/2021/day/4
### Part 1

### Part 2

## Day 3
https://adventofcode.com/2021/day/3
### Part 1

### Part 2

## Day 2
https://adventofcode.com/2021/day/2
### Part 1

### Part 2

## Day 1
https://adventofcode.com/2021/day/1
### Part 1

### Part 2